Type checking via a decorator.

In [1]:

import pandas as pd
from type_signature import TypeSignature

In [2]:
@TypeSignature(
        int, int, 
        c=pd.DataFrame({'x': [int]}),
        return_type=pd.DataFrame({'z': [float]}))
def fn(a, b, *, c, d=None):
    """doc"""
    return d

In [3]:
help(fn)

Help on function fn in module __main__:

fn(a, b, *, c, d=None)
    arg types:
     positional args
    [<class 'int'>, <class 'int'>]
     named args
    {'c':                  x
    0  {<class 'int'>}}
     return type
                       z
    0  {<class 'float'>}
    
    
    doc



In [4]:
try:
    fn(1, 2, c=3)
except ValueError as e:
    print(e)

fn arg c  should be type pandas.DataFrame, was type int


In [5]:
try:
    fn(1, 2, c=pd.DataFrame({'z': [7]}))
except ValueError as e:
    print(e)

fn arg c  missing required column 'x'


In [6]:
fn(1, 2, c=pd.DataFrame({'x': [3]}), d=pd.DataFrame({'z': [7.0]}))

,z
0,7.0


In [7]:
try:
    fn(1, 2, c=pd.DataFrame({'x': [3.0]}))
except ValueError as e:
    print(e)

fn arg c  column 'x' has a type float entry, when one of {int} expected


In [8]:
try:
    fn(1, 2, c=pd.DataFrame({'x': [30], "z": [17.2]}), d=pd.DataFrame({'q': [7.0]}))
except ValueError as e:
    print(e)

return value:  missing required column 'z'
